In [76]:
import pandas as pd
import numpy as np
import transformers
from datasets import Dataset

In [77]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


# load dataset

In [78]:
data = pd.read_csv('new2.csv')

In [79]:
data.head(6)

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
5,5,"every break, you must be nervous, like somethi...",Anxiety


In [80]:
data.drop(columns=["Unnamed: 0"],axis=1,inplace=True)

# preprocessing


In [81]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

sw=set(stopwords.words('english'))

def preprocess_text(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
     
    words=text.split()  
    words=[word for word in words if word not in sw]
    
    return " ".join(words)


data["statement"]=data["statement"].fillna("").apply(preprocess_text)

preprocess_text("hi MY name is aryan my hobby is plraying croicket")

<>:10: SyntaxWarning: invalid escape sequence '\['
<>:10: SyntaxWarning: invalid escape sequence '\['
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20492\1256883032.py:10: SyntaxWarning: invalid escape sequence '\['
  text=re.sub('\[.*?\]','',text)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'hi name aryan hobby plraying croicket'

In [82]:
data["status"].value_counts()

status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64

In [83]:
from sklearn.preprocessing import LabelEncoder


label=LabelEncoder()

In [84]:
data["label"]=label.fit_transform(data['status'])

In [85]:
x=data["statement"]
y=data["status"]

In [86]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
x_train.shape

(42434,)

## tokenization

In [90]:
from transformers import BertTokenizer,BertForSequenceClassification,Trainer,TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
train_encoding=tokenizer(list(x),padding=True,truncation=True,max_length=200)
test_encoding=tokenizer(list(y),padding=True,truncation=True,max_length=200)

In [110]:
train_dataset=Dataset.from_dict({"input_ids":train_encoding["input_ids"], 'attention_mask':train_encoding['attention_mask'],"label":x.tolist()})

test_dataset=Dataset.from_dict({"input_ids":test_encoding["input_ids"], 'attention_mask':test_encoding['attention_mask'],"label":y.tolist()})

In [112]:
test_dataset[0]

{'input_ids': [101, 10089, 102, 0],
 'attention_mask': [1, 1, 1, 0],
 'label': 'Anxiety'}

In [123]:
import transformers
print(transformers.__version__)

from transformers import TrainingArguments



4.57.1


In [126]:
from transformers import TrainingArguments
print("TrainingArguments module:", TrainingArguments.__module__)
print("TrainingArguments file:", TrainingArguments.__file__)


TrainingArguments module: transformers.training_args


AttributeError: type object 'TrainingArguments' has no attribute '__file__'

# fine tuning


In [125]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=3
)



TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
model= BertForSequenceClassification.from_pretrained('bert-base-uncased')
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    
    
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'str' object has no attribute 'batch_eval_metrics'